## Introduction

In this lab assessment, you'll practice your knowledge of JOIN statements and subqueries, using various types of joins and various methods for specifying the links between them. One of the main benefits of using a relational database is the table relations that define them which allow you to access and connect data together via shared columns. By writing more advanced SQL queries that utilize joins and subqueries you can provide a deeper and more granular level of analysis and data retrieval.

This assessment will continue looking at the familiar Northwind database that contains customer relationship management (CRM) data as well as employee and product data. You will take a deeper dive into this database in order to accomplish more advanced SQL queries that require you to access data from multiple tables at once. 

Imagine that you are working in an analyst role for the sales rep team. They have collaborated with the customer relations and the product teams to take a comprehensive look at the employee to customer pipeline in an attempt to find areas of improvement and potential growth. You have been asked to provide some specific data and statistics regarding this project.

## Learning Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Choose and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Database

The database will be the customer relationship management (CRM) database, which has the following ERD.

![Database-Schema.png](ERD.png)

### Connect to the database

In the cell below we have provided the code to import both pandas and sqlite3 as well as define and create the connection to the database you will use. Also displayed is the schema and table names from the database. Use this information in conjunction with the ERD image above to assist in creating your SQL Queries.

Major Hint: Look for the shared columns across tables you need to 'join' together.

In [1]:
# CodeGrade step0
# Run this cell without changes

# SQL Library and Pandas Library
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.sqlite')

pd.read_sql("""SELECT * FROM sqlite_master""", conn)

# employees = pd.read_sql("""SELECT * FROM employees""", conn)
# print(employees)
# offices = pd.read_sql("""SELECT * FROM offices""", conn)
# print(offices)


,type,name,tbl_name,rootpage,sql
0,table,orderdetails,orderdetails,2,"CREATE TABLE `orderdetails` (`orderNumber`, `p..."
1,table,payments,payments,28,"CREATE TABLE `payments` (`customerNumber`, `ch..."
2,table,offices,offices,32,"CREATE TABLE `offices` (`officeCode`, `city`, ..."
3,table,customers,customers,33,"CREATE TABLE `customers` (`customerNumber`, `c..."
4,table,orders,orders,38,"CREATE TABLE `orders` (`orderNumber`, `orderDa..."
5,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
6,table,products,products,47,"CREATE TABLE `products` (`productCode`, `produ..."
7,table,employees,employees,56,"CREATE TABLE `employees` (`employeeNumber`, `l..."


## Part 1: Join and Filter

### Step 1

The company would like to let Boston employees go remote but need to know more information about who is working in that office. Return the first and last names and the job titles for all employees in Boston.

In [2]:
# CodeGrade step1
# Replace None with your code

df_boston = pd.read_sql("""
SELECT firstName, lastName FROM employees
JOIN offices
    USING(officeCode)
WHERE city = "Boston"
""", conn)

print(df_boston)

  firstName   lastName
0     Julie   Firrelli
1     Steve  Patterson


### Step 2

Recent downsizing and employee attrition have caused some mixups in office tracking and the company is worried they are supporting a 'ghost' location. Are there any offices that have zero employees?

In [3]:
# CodeGrade step2
# Replace None with your code

df_zero_emp = pd.read_sql("""
SELECT o.officeCode, o.city,
       COUNT(e.employeeNumber) AS num_employees                         
FROM offices AS o
LEFT JOIN employees AS e
    ON o.officeCode = e.officeCode
GROUP BY o.officeCode, o.city
HAVING num_employees = 0;
""", conn)
print(df_zero_emp)

Empty DataFrame
Columns: [officeCode, city, num_employees]
Index: []


## Part 2: Type of Join

### Step 3

As a part of this larger analysis project the HR department is taking the time to audit employee records to make sure nothing is out of place and have asked you to produce a report of all employees. Return the employees first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [4]:
# CodeGrade step3
# Replace None with your code
df_employee = pd.read_sql("""
SELECT e.firstName, e.lastName,
        o.city, o.state
FROM employees AS e
LEFT JOIN offices AS o
    USING(officeCode)
ORDER BY firstname ASC,  e.lastName ASC;                    
""", conn)

print(df_employee)

   firstName   lastName           city       state
0       Andy     Fixter         Sydney            
1    Anthony        Bow  San Francisco          CA
2      Barry      Jones         London            
3      Diane     Murphy  San Francisco          CA
4   Foon Yue      Tseng            NYC          NY
5     George     Vanauf            NYC          NY
6     Gerard     Bondur          Paris            
7     Gerard  Hernandez          Paris            
8       Jeff   Firrelli  San Francisco          CA
9      Julie   Firrelli         Boston          MA
10     Larry       Bott         London            
11    Leslie   Jennings  San Francisco          CA
12    Leslie   Thompson  San Francisco          CA
13      Loui     Bondur          Paris            
14      Mami      Nishi          Tokyo  Chiyoda-Ku
15    Martin     Gerard          Paris            
16      Mary  Patterson  San Francisco          CA
17    Pamela   Castillo          Paris            
18     Peter      Marsh        

### Step 4
The customer management and sales rep team know that they have several 'customers' in the system that have not placed any orders. They want to reach out to these customers with updated product catalogs to try and get them to place initial orders. Return all of the customer's contact information (first name, last name, and phone number) as well as their sales rep's employee number for any customer that has not placed an order. Sort the results alphabetically based on the contact's last name

There are several approaches you could take here, including a left join and filtering on null values or using a subquery to filter out customers who do have orders. In total there are 24 customers who have not placed an order.

In [5]:
# CodeGrade step4
# Replace None with your code
df_contacts = pd.read_sql("""
SELECT c.contactLastName, c.contactFirstName, c.Phone, c.salesRepEmployeeNumber
FROM customers AS c
LEFT JOIN orders AS o
    USING(customerNumber)
WHERE o.orderNumber is Null
ORDER BY c.contactLastName ASC;                       
""", conn)

print(df_contacts)

    contactLastName contactFirstName             phone salesRepEmployeeNumber
0       Altagar,G M           Raanan  + 972 9 959 8555                       
1          Andersen              Mel       030-0074555                       
2             Anton           Carmen    +34 913 728555                       
3            Camino       Alejandra      (91) 745 6555                       
4            Cramer          Philip         0555-09555                       
5             Feuer       Alexander        0342-555176                       
6            Franco            Keith        2035557845                   1286
7           Franken           Peter        089-0877555                       
8          Harrison               Ed  +41 26 425 50 01                       
9           Josephs            Karin       0251-555259                       
10            Kloss           Horst        0372-555188                       
11            Kuger           Armand   +27 21 550 3555          

## Part 3: Built-in Function

### Step 5

The accounting team is auditing their figures and wants to make sure all customer payments are in alignment, they have asked you to produce a report of all the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. They have asked that these results be sorted in descending order by the payment amount.

Hint: A member of their team mentioned that they are not sure the 'amount' column is being stored as the right datatype so keep this in mind when sorting.

In [6]:
# CodeGrade step5
# Replace None with your code
df_payment = pd.read_sql("""
SELECT c.contactLastName, c.contactFirstName, 
    p.paymentDate, p.amount                   
FROM customers AS c
JOIN payments AS p
    USING(customerNumber)
ORDER BY CAST(p.amount AS REAL) DESC;                        
""", conn)

print(df_payment)

    contactLastName contactFirstName paymentDate     amount
0            Freyre           Diego   2005-03-18  120166.58
1            Freyre           Diego   2004-12-31  116208.40
2            Nelson            Susan  2003-08-15  111654.40
3         Natividad             Eric  2003-12-26  105743.00
4            Nelson            Susan  2005-03-05  101244.59
..              ...              ...         ...        ...
268         Schmitt          Carine   2004-12-18    1676.14
269        Cartrain         Pascale   2003-04-19    1627.56
270      Bergulfsen           Jonas   2003-10-28    1491.38
271        Cartrain         Pascale   2003-08-22    1128.20
272       Shimamura            Akiko  2005-05-18     615.45

[273 rows x 4 columns]


## Part 4: Joining and Grouping

### Step 6

The sales rep team has noticed several key team members that stand out as having trustworthy business relations with their customers, reflected by high credit limits indicating more potential for orders. The team wants you to identify these 4 individuals. Return the employee number, first name, last name, and number of customers for employees whose customers have an average credit limit over 90k. Sort by number of customers from high to low.

In [8]:
# CodeGrade step6
# Replace None with your code

df_credit = pd.read_sql("""
SELECT
    e.employeeNumber, e.firstName, e.lastName,
    AVG(c.creditLimit) AS avg_creditLimit,
    COUNT(c.customerNUmber) AS customer_count
FROM employees AS e
JOIN customers AS c
    ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY e.employeeNumber
HAVING avg_creditLimit > 90000
ORDER BY customer_count DESC
LIMIT 4
""", conn)

print(df_credit)

  employeeNumber firstName   lastName  avg_creditLimit  customer_count
0           1501     Larry       Bott     91187.500000               8
1           1370    Gerard  Hernandez     91785.714286               7
2           1165    Leslie   Jennings    100433.333333               6
3           1612     Peter      Marsh     92080.000000               5


### Step 7

The product team is looking to create new model kits and wants to know which current products are selling the most in order to get an idea of what is popular. Return the product name and count the number of orders for each product as a column named 'numorders'. Also return a new column, 'totalunits', that sums up the total quantity of product sold (use the quantityOrdered column). Sort the results by the totalunits column, highest to lowest, to showcase the top selling products.

In [9]:
# CodeGrade step7
# Replace None with your code
df_product_sold = pd.read_sql("""
SELECT p.productName, 
        COUNT(DISTINCT od.orderNumber) AS numorders,
        SUM(od.quantityOrdered) AS totalunits                              
FROM products AS p
JOIN orderdetails AS od
    USING(productCode)
GROUP by productName
ORDER by totalunits DESC                            
""", conn)

print(df_product_sold)

                                 productName  numorders  totalunits
0                1992 Ferrari 360 Spider red         53        1808
1                       1937 Lincoln Berline         28        1111
2                  American Airlines: MD-11S         28        1085
3    1941 Chevrolet Special Deluxe Cabriolet         28        1076
4               1930 Buick Marquette Phaeton         28        1074
..                                       ...        ...         ...
104             1999 Indy 500 Monte Carlo SS         25         855
105                       1911 Ford Town Car         25         832
106         1936 Mercedes Benz 500k Roadster         25         824
107               1970 Chevy Chevelle SS 454         25         803
108                    1957 Ford Thunderbird         24         767

[109 rows x 3 columns]


## Part 5: Multiple Joins

### Step 8

As a follow-up to the above question, the product team also wants to know how many different customers ordered each product to get an idea of market reach. Return the product name, code, and the total number of customers who have ordered each product, aliased as 'numpurchasers'. Sort the results by the highest  number of purchasers.

Hint: You might need to join more than 2 tables. Use DISTINCT to return unique/different values.

In [10]:
# CodeGrade step8
# Replace None with your code
df_total_customers = pd.read_sql("""
SELECT p.productName, p.productCode, 
        COUNT(DISTINCT o.customerNumber) AS numpurchasers
FROM products AS p
JOIN orderdetails AS od
    USING(productCode)
JOIN orders AS o
    USING(orderNumber)
GROUP BY p.productCode, p.productName
ORDER BY numpurchasers DESC;
""", conn)

print(df_total_customers)

                             productName productCode  numpurchasers
0            1992 Ferrari 360 Spider red    S18_3232             40
1               1952 Alpine Renault 1300    S10_1949             27
2                    1972 Alfa Romeo GTA    S10_4757             27
3                     1934 Ford V8 Coupe    S18_2957             27
4                       Boeing X-32A JSF    S72_1253             27
..                                   ...         ...            ...
104  1958 Chevy Corvette Limited Edition    S24_2840             19
105                   1949 Jaguar XK 120    S24_2766             18
106                    1952 Citroen-15CV    S24_2887             18
107            1969 Chevrolet Camaro Z28    S24_3191             18
108                  2002 Chevy Corvette    S24_3432             18

[109 rows x 3 columns]


### Step 9

The custom relations team is worried they are not staffing locations properly to account for customer volume. They want to know how many customers there are per office. Return the count as a column named 'n_customers'. Also return the office code and city.

In [11]:
# CodeGrade step9
# Replace None with your code
df_customers = pd.read_sql("""
SELECT o.officeCode, o.city, COUNT(c.customerNumber) AS n_customers
FROM customers AS c
JOIN employees AS e
    ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN offices AS o
    USING(officeCode)
GROUP BY o.officeCode, o.city
ORDER BY n_customers DESC;                 
""", conn)

print(df_customers)

  officeCode           city  n_customers
0          4          Paris           29
1          7         London           17
2          3            NYC           15
3          1  San Francisco           12
4          2         Boston           12
5          6         Sydney           10
6          5          Tokyo            5


## Part 6: Subquery

### Step 10

Having looked at the results from above, the product team is curious to dig into the underperforming products. They want to ask members of the team who have sold these products about what kind of messaging was successful in getting a customer to buy these specific products. Using a subquery or common table expression (CTE), select the employee number, first name, last name, city of the office, and the office code for employees who sold products that have been ordered by fewer than 20 customers.

Hint: Start with the subquery, find all the products that have been ordered by 19 or less customers, consider adapting one of your previous queries.

In [17]:
# CodeGrade step10
# Replace None with your code
df_under_20 = df_under_20 = pd.read_sql("""
WITH underperform_product AS (
    SELECT productCode 
    FROM orderdetails
    JOIN orders ON orderdetails.orderNumber = orders.orderNumber
    GROUP BY orderdetails.productCode
    HAVING COUNT(DISTINCT orders.customerNumber) <= 19                                   
)
                                    
SELECT 
    employees.employeeNumber,
    employees.firstName,
    employees.lastName,
    offices.city,                                                                                                           
    offices.officeCode
FROM employees
JOIN offices ON offices.officeCode = employees.officeCode
JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber
JOIN orders ON orders.customerNumber = customers.customerNumber
JOIN orderdetails ON orderdetails.orderNumber = orders.orderNumber
WHERE orderdetails.productCode IN (
    SELECT productCode FROM underperform_product
)
GROUP BY employees.employeeNumber
ORDER BY employees.employeeNumber;                                                                                                                                                                                                 
""", conn)

print(df_under_20)

   employeeNumber firstName   lastName           city officeCode
0            1165    Leslie   Jennings  San Francisco          1
1            1166    Leslie   Thompson  San Francisco          1
2            1188     Julie   Firrelli         Boston          2
3            1216     Steve  Patterson         Boston          2
4            1286  Foon Yue      Tseng            NYC          3
5            1323    George     Vanauf            NYC          3
6            1337      Loui     Bondur          Paris          4
7            1370    Gerard  Hernandez          Paris          4
8            1401    Pamela   Castillo          Paris          4
9            1501     Larry       Bott         London          7
10           1504     Barry      Jones         London          7
11           1611      Andy     Fixter         Sydney          6
12           1612     Peter      Marsh         Sydney          6
13           1621      Mami      Nishi          Tokyo          5
14           1702    Mart

### Close the connection

In [18]:
# Run this cell without changes

conn.close()